Clickhouse documentation chat

Iterate all .md files in the directory recursively: /Users/brainco/BrainCo/ML/clickhouse-docs

In [30]:
import os
import fnmatch

dir_path = '/Users/brainco/BrainCo/ML/clickhouse/docs' # replace with the actual path to the directory containing .md files
md_files = []
for subdir, dirs, files in os.walk(dir_path):
    for file in files:
        if fnmatch.fnmatch(file, '*.md'):
            filepath = os.path.join(subdir, file)
            md_files.append(filepath)

print(f'Number of .md files: {len(md_files)}')

Number of .md files: 1717


Load all markdown files using langchain's Makrdown loader and split them into document objects using `MarkdownTextSplitter`.

In [31]:
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import MarkdownTextSplitter


splitter = MarkdownTextSplitter(chunk_size=1000, chunk_overlap=150)

# Load the .md files into a list of Document objects
# All docs
all_docs = []
for md_file in md_files:
    loader = UnstructuredMarkdownLoader(md_file)
    data = loader.load()
    docs = splitter.split_documents(data)
    # Append every doc in docs to all_docs
    for doc in docs:
        all_docs.append(doc)

print(f'Number of documents: {len(all_docs)}')


Number of documents: 11246


In [ ]:
pip3 install pinecone-client

Load the documents into Pinecone using OpenAI's embedding model, you only need to do this once.

In [ ]:
from langchain.vectorstores import Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone
import os

# initialize pinecone
pinecone.init(
    api_key=os.environ['PINECONE_API_KEY'],  # find at app.pinecone.io
    environment="us-west1-gcp-free"  # next to api key in console
)

index_name = "clickhouse-docs"
embeddings = OpenAIEmbeddings()
docsearch = Pinecone.from_documents(all_docs, embeddings, index_name=index_name)


# if you already have an index, you can load it like this
# docsearch = Pinecone.from_existing_index(index_name, embeddings)

# query = "arrayDifference()"
# docs = docsearch.similarity_search(query)

Testing the pinecone query.

In [33]:
query = "array join usage"
docs = docsearch.similarity_search(query)
print(docs)

[Document(page_content="The arrayJoin function affects all sections of the query, including the WHERE section. Notice the result 2, even though the subquery returned 1 row.\n\nExample:\n\nsql\nSELECT sum(1) AS impressions\nFROM\n(\n    SELECT ['Istanbul', 'Berlin', 'Bobruisk'] AS cities\n)\nWHERE arrayJoin(cities) IN ['Istanbul', 'Berlin'];\n\ntext\n┌─impressions─┐\n│           2 │\n└─────────────┘\n\nA query can use multiple arrayJoin functions. In this case, the transformation is performed multiple times and the rows are multiplied.\n\nExample:\n\nsql\nSELECT\n    sum(1) AS impressions,\n    arrayJoin(cities) AS city,\n    arrayJoin(browsers) AS browser\nFROM\n(\n    SELECT\n        ['Istanbul', 'Berlin', 'Bobruisk'] AS cities,\n        ['Firefox', 'Chrome', 'Chrome'] AS browsers\n)\nGROUP BY\n    2,\n    3", metadata={'source': '/Users/brainco/BrainCo/ML/clickhouse/docs/en/sql-reference/functions/array-join.md'}), Document(page_content='slug: /en/sql-reference/statements/select/arra

This is a simple demonstration of querying the language model of our own choice with the custom knowledge base we built previously. 
- Firstly, establish the connection to pinecone.
- Then using langchain's `load_qa_with_sources_chain()` to ask the knowledge base a question.

In [12]:
from langchain.vectorstores import Pinecone
from langchain.chains.llm import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.memory import ConversationBufferMemory
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts.prompt import PromptTemplate
import pinecone
import os

# Replace with your own OpenAI API key
os.environ['OPENAI_API_KEY']='sk-xxx'
os.environ['PINECONE_API_KEY'] = 'xxx'


# initialize pinecone
pinecone.init(
    api_key=os.environ['PINECONE_API_KEY'],  # find at app.pinecone.io
    environment="us-west1-gcp-free"  # next to api key in console
)


index_name = "clickhouse-docs"
docsearch = Pinecone.from_existing_index(embedding=OpenAIEmbeddings(), index_name=index_name)

# llm = OpenAI(model_name='gpt-3.5-turbo', temperature=0.7)

_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question.
When your response contains code, format them into a markdown code block.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""

llm = OpenAI()

question_generator = LLMChain(llm=llm, prompt=PromptTemplate.from_template(_template))

# memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
doc_chain = load_qa_with_sources_chain(llm, chain_type="map_reduce")

qa_with_source = ConversationalRetrievalChain(
    retriever=docsearch.as_retriever(),
    question_generator=question_generator,
    combine_docs_chain=doc_chain
)

chat_history = []
query3 = "How to use arrayJoin(), please give an code example"
result = qa_with_source({'question': query3, 'chat_history': chat_history})
print(result)


{'question': 'How to use arrayJoin(), please give an code example', 'chat_history': [], 'answer': " arrayJoin() takes an array as an argument, and propagates the source row to multiple rows for the number of elements in the array. All the values in columns are simply copied, except the values in the column where this function is applied; it is replaced with the corresponding array value.\n\nExample: \n\nsql\nSELECT arrayJoin([1, 2, 3] AS src) AS dst, 'Hello', src\nSOURCES:\n/Users/brainco/BrainCo/ML/clickhouse/docs/en/sql-reference/functions/array-join.md\n/Users/brainco/BrainCo/ML/clickhouse/docs/zh/sql-reference/functions/array-join.md\n/Users/brainco/BrainCo/ML/clickhouse/docs/zh/sql-reference/statements/select/array-join.md"}


In [13]:
query3 = "How to use retention(), what can it potentially do?"
result = qa_with_source({'question': query3, 'chat_history': chat_history})
print(result)

{'question': 'How to use retention(), what can it potentially do?', 'chat_history': [], 'answer': ' retention() is an expression in ClickHouse that returns a UInt8 result (1 or 0) which indicates whether a condition was met for an event or not.\nSOURCES: /Users/brainco/BrainCo/ML/clickhouse/docs/en/sql-reference/aggregate-functions/parametric-functions.md'}
